In [1]:
import os
import sys
from petsc4py import PETSc

print(f"PETSc verze: {PETSc.Sys.getVersion()}")
print(f"Cesta k PETSc: {os.environ.get('PETSC_DIR')}")
print(f"Architektura: {os.environ.get('PETSC_ARCH')}")

PETSc verze: (3, 24, 4)
Cesta k PETSc: /home/tompaholik/petsc
Architektura: arch-linux-c-opt
